In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

x = 10

## Summary Statistics function

In [37]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

def summary_statistics_g(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'saves': 'sum',
        'shots': 'sum',
        'start': 'sum',
        'shutout': 'sum',
        'fantasyPoints': 'sum'
    }).reset_index()
    
    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)
    
    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

# Pull in the data files from scrape

In [40]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# file_name = f"data/manual-adjustments.xlsx"
rankings = pd.read_excel(file_name, index_col=False)

# fileName = f"data/summary_stats_last{x}-{today.strftime('%Y-%m-%d')}.csv"
# last_10 = pd.read_csv(fileName, index_col=False)

In [20]:
ordercolumns = rankings.columns.to_list()
ordercolumns

['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-11-27',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01']

## Code to re-tag rankings for save

In [17]:
# file_name = f"data/manual-adjustments.xlsx"
# manual = pd.read_excel(file_name)
# for index, row in rankings.iterrows():
#     try:
#         rankings.at[index, 'rank_2023-11-27'] = player_bios_adjusted.loc[player_bios_adjusted['espnId'] == row['espnId'], 'rank'].iloc[0]
#     except:
#         rankings.at[index, 'rank_2023-11-27'] = 800

In [19]:
# fileName = f"data/rankings_worksheet_2023-11-27.xlsx"
# rankings.to_excel(fileName, index=False)

## New column order

In [37]:
column_order = ['espnId',
 'FullName',
 'fantasyRemain',
 'rank_2023-11-27',
 'rank_2023-11-20',
 'rank_2023-11-13',
 'rank_2023-11-06',
 'rank_2023-10-30',
 'rank_2023-10-23',
 'rank_2023-09-01'
]

In [34]:
player_bios

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,adjFantasyRemain
0,8476453,N. Kucherov,TBL,R,25780,20,73.5,10.26,3.68,0,...,Nikita Kucherov,1993-06-17,L,71,182,99.876999,RW,2563060,4.59,242.983333
1,8475660,C. Talbot,LAK,G,49933,14,63.2,4.56,4.51,0,...,Cam Talbot,1987-07-05,L,76,200,80.380712,G,5734,4.59,NaN
2,8480069,C. Makar,COL,D,28971,20,65.4,8.13,3.27,0,...,Cale Makar,1998-10-30,R,71,187,99.907738,D,4233563,NaN,NaN
3,8477956,D. Pastrnak,BOS,R,23219,20,64.7,10.03,3.24,0,...,David Pastrnak,1996-05-25,R,72,196,99.910678,RW,3114778,NaN,NaN
4,8479318,A. Matthews,TOR,C,24219,19,60.0,8.92,3.16,0,...,Auston Matthews,1997-09-17,L,75,215,99.913606,C,4024123,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,8480992,M. Chrona,SJS,G,1830,1,-5.4,-10.62,-5.40,0,...,Magnus Chrona,2000-08-28,L,76,194,0.016133,G,0,NaN,NaN
817,8479406,F. Gustavsson,MIN,G,36534,11,-7.8,-0.77,-0.71,0,...,Filip Gustavsson,1998-06-07,L,74,184,64.863240,G,4272674,NaN,NaN
818,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,Jack Campbell,1992-01-09,L,75,200,2.566547,G,5473,NaN,NaN
819,8476341,A. Forsberg,OTT,G,22557,7,-7.0,-1.12,-1.00,0,...,Anton Forsberg,1992-11-27,L,75,195,1.529527,G,3036851,NaN,NaN


# Adjusted Fantasy Remain, done with Last 10

In [41]:
for index, row in player_bios.iterrows():
    if row['position'] != 'G':
        player_log = all_df.loc[all_df['playerId'] == row['playerId']]
        player_log.sort_values('gameDate', inplace=True, ascending=False)
        player_log = player_log.head(10)
        last_10_sum = summary_statistics(player_log)
        try:
            last_10_FPPG = last_10_sum['FPPG'].iloc[0]
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'adjFantasyRemain'] = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
        except:
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
    else:
        player_log = allG_df.loc[allG_df['playerId'] == row['playerId']]
        player_log.sort_values('gameDate', inplace=True, ascending=False)
        player_log = player_log.head(10)
        last_10_sum = summary_statistics_g(player_log)
        try:
            last_10_FPPG = last_10_sum['FPP60'].iloc[0]
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'adjFantasyRemain'] = (((row['FPP60'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
        except:
            player_bios.at[index, 'last10FPPG'] = last_10_FPPG
            player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']

player_bios.sort_values('adjFantasyRemain')

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,adjFantasyRemain
0,8476453,N. Kucherov,TBL,R,25780,20,73.5,10.26,3.68,0,...,Nikita Kucherov,1993-06-17,L,71,182,99.878611,RW,2563060,4.59,242.983333
1,8475660,C. Talbot,LAK,G,49933,14,63.2,4.56,4.51,0,...,Cam Talbot,1987-07-05,L,76,200,80.821396,G,5734,5.81,223.950000
2,8480069,C. Makar,COL,D,28971,20,65.4,8.13,3.27,0,...,Cale Makar,1998-10-30,R,71,187,99.904926,D,4233563,3.71,211.833333
3,8477956,D. Pastrnak,BOS,R,23219,20,64.7,10.03,3.24,0,...,David Pastrnak,1996-05-25,R,72,196,99.910786,RW,3114778,3.34,202.946667
4,8479318,A. Matthews,TOR,C,24219,19,60.0,8.92,3.16,0,...,Auston Matthews,1997-09-17,L,75,215,99.907861,C,4024123,3.21,200.130000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,8480992,M. Chrona,SJS,G,1830,1,-5.4,-10.62,-5.40,0,...,Magnus Chrona,2000-08-28,L,76,194,0.016112,G,0,-10.62,-10.620000
817,8479406,F. Gustavsson,MIN,G,36534,11,-7.8,-0.77,-0.71,0,...,Filip Gustavsson,1998-06-07,L,74,184,64.259140,G,4272674,-2.51,-44.550000
818,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,Jack Campbell,1992-01-09,L,75,200,2.541305,G,5473,-1.93,-25.090000
819,8476341,A. Forsberg,OTT,G,22557,7,-7.0,-1.12,-1.00,0,...,Anton Forsberg,1992-11-27,L,75,195,1.485127,G,3036851,-1.12,-28.000000


In [44]:
player_bios.sort_values('adjFantasyRemain', ascending=False).head(50)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,adjFantasyRemain
0,8476453,N. Kucherov,TBL,R,25780,20,73.5,10.26,3.68,0,...,Nikita Kucherov,1993-06-17,L,71,182,99.878611,RW,2563060,4.59,242.983333
1,8475660,C. Talbot,LAK,G,49933,14,63.2,4.56,4.51,0,...,Cam Talbot,1987-07-05,L,76,200,80.821396,G,5734,5.81,223.950000
2,8480069,C. Makar,COL,D,28971,20,65.4,8.13,3.27,0,...,Cale Makar,1998-10-30,R,71,187,99.904926,D,4233563,3.71,211.833333
3,8477956,D. Pastrnak,BOS,R,23219,20,64.7,10.03,3.24,0,...,David Pastrnak,1996-05-25,R,72,196,99.910786,RW,3114778,3.34,202.946667
4,8479318,A. Matthews,TOR,C,24219,19,60.0,8.92,3.16,0,...,Auston Matthews,1997-09-17,L,75,215,99.907861,C,4024123,3.21,200.130000
6,8476468,J. Miller,VAN,C,26486,22,71.9,9.77,3.27,0,...,J.T. Miller,1993-03-14,L,73,218,99.725026,C,2590852,3.35,197.800000
7,8477939,W. Nylander,TOR,R,23026,19,57.4,8.97,3.02,0,...,William Nylander,1996-05-01,R,72,202,99.831810,RW,3114736,2.98,189.420000
10,8477492,N. MacKinnon,COL,C,26312,20,58.3,7.98,2.92,0,...,Nathan MacKinnon,1995-09-01,R,72,200,99.900541,C,3041969,3.27,188.273333
16,8476887,F. Forsberg,NSH,L,23028,20,55.4,8.66,2.77,0,...,Filip Forsberg,1994-08-13,R,73,205,89.657745,LW,2968772,3.54,187.653333
11,8478402,C. McDavid,EDM,C,23373,18,52.3,8.06,2.91,0,...,Connor McDavid,1997-01-13,L,73,194,99.875676,C,3895074,3.19,186.206667


# Get adjustment to Fantasy Remain

In [45]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [71]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)
opp_defense

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,144613,54,13,190,98,180,208.8,21,20,9.94,8,Anaheim Ducks
1,ARI,139111,37,14,185,147,172,189.7,20,9,9.48,12,Arizona Coyotes
2,BOS,137690,27,5,170,147,194,170.7,20,9,8.54,24,Boston Bruins
3,BUF,146279,33,13,165,131,196,190.1,21,6,9.05,19,Buffalo Sabres
4,CAR,138886,29,9,126,118,203,178.4,20,9,8.92,20,Carolina Hurricanes
5,CBJ,154226,39,11,200,96,244,216.6,22,7,9.85,10,Columbus Blue Jackets
6,CGY,147680,39,12,173,138,218,208.1,21,8,9.91,9,Calgary Flames
7,CHI,132228,39,11,169,109,177,182.8,19,10,9.62,11,Chicago Blackhawks
8,COL,137388,28,6,146,135,200,172.6,20,7,8.63,22,Colorado Avalanche
9,DAL,132499,30,8,170,85,183,167.0,19,5,8.79,21,Dallas Stars


In [55]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,75460,55,604,50.8,21,448,487,0,80,94,2.42,13,Anaheim Ducks
1,ARI,72101,61,542,23.2,20,396,439,1,77,95,1.16,20,Arizona Coyotes
2,BOS,71184,64,624,10.0,20,437,486,0,102,117,0.50,28,Boston Bruins
3,BUF,75360,55,580,47.0,21,427,471,1,85,95,2.24,15,Buffalo Sabres
4,CAR,72368,64,670,32.2,20,501,546,1,86,101,1.61,18,Carolina Hurricanes
5,CBJ,79246,60,661,71.2,22,488,541,2,99,106,3.24,4,Columbus Blue Jackets
6,CGY,76011,56,666,66.0,21,516,561,1,80,88,3.14,5,Calgary Flames
7,CHI,67964,46,523,63.4,19,403,443,2,57,63,3.34,3,Chicago Blackhawks
8,COL,71441,68,646,13.6,20,450,497,3,109,125,0.68,26,Colorado Avalanche
9,DAL,68464,61,582,16.2,19,423,467,1,79,92,0.85,24,Dallas Stars


In [53]:
remainSked = homeOnlySked.loc[homeOnlySked['gameDT'] >= str(datetime.utcnow())]
remainSked.sort_values('gameDate', inplace=True)
remainSked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
7,2023020322,2023-11-27,Monday 07:00 PM,BUF,NYR,2023-11-28 00:00:00,0
214,2023020324,2023-11-27,Monday 09:00 PM,TBL,COL,2023-11-28 02:00:00,0
1283,2023020326,2023-11-27,Monday 10:30 PM,WSH,SJS,2023-11-28 03:30:00,0
1118,2023020321,2023-11-27,Monday 07:00 PM,BOS,CBJ,2023-11-28 00:00:00,0
1078,2023020323,2023-11-27,Monday 07:00 PM,FLA,OTT,2023-11-28 00:00:00,0
...,...,...,...,...,...,...,...
1229,2023021307,2024-04-18,Thursday 07:00 PM,SEA,MIN,2024-04-18 23:00:00,0
163,2023021312,2024-04-18,Thursday 10:30 PM,CHI,LAK,2024-04-19 02:30:00,0
327,2023021308,2024-04-18,Thursday 08:00 PM,VAN,WPG,2024-04-19 00:00:00,0
1024,2023021309,2024-04-18,Thursday 09:00 PM,SJS,CGY,2024-04-19 01:00:00,0


In [99]:
team_names_FPAPG = team_names.copy()
team_names_FPAPG['G_FPAPG'] = 0
team_names_FPAPG['F_FPAPG'] = 0
team_names_FPAPG['D_FPAPG'] = 0

for index, row in remainSked.iterrows():
    index_of_away = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['awayTeam']].index[0]
    index_of_home = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['homeTeam']].index[0]
    
    tempAwayG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeG = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'G_FPAPG'] += tempAwayG
    team_names_FPAPG.at[index_of_home, 'G_FPAPG'] += tempHomeG
    
    tempAwayF = opp_forward.loc[opp_forward['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeF = opp_forward.loc[opp_forward['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'F_FPAPG'] += tempAwayF
    team_names_FPAPG.at[index_of_home, 'F_FPAPG'] += tempHomeF
    
    tempAwayD = opp_defense.loc[opp_defense['abbreviation'] == row['homeTeam']]['FPAPG'].iloc[0]
    tempHomeD = opp_defense.loc[opp_defense['abbreviation'] == row['awayTeam']]['FPAPG'].iloc[0]
    team_names_FPAPG.at[index_of_away, 'D_FPAPG'] += tempAwayD
    team_names_FPAPG.at[index_of_home, 'D_FPAPG'] += tempHomeD
    
# Gmean = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Fmean = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Dmean = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).mean()
# Gstd = (team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Fstd = (team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']).std()
# Dstd = (team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']).std()

# for index, row in team_names_FPAPG.iterrows():
#     Gmodifier = (row['G_FPAPG']/row['gamesRemaining']) - Gmean
#     adjModifier = (Gmodifier - Gmean) / Gstd
    
#     team_names_FPAPG.at[index, 'Gsos'] = adjModifier
    
#     Fmodifier = (row['F_FPAPG']/row['gamesRemaining']) - Fmean
#     adjModifier = (Fmodifier - Fmean) / Fstd
    
#     team_names_FPAPG.at[index, 'Fsos'] = adjModifier
    
#     Dmodifier = (row['D_FPAPG']/row['gamesRemaining']) - Dmean
#     adjModifier = (Dmodifier - Dmean) / Dstd
    
#     team_names_FPAPG.at[index, 'Dsos'] = adjModifier
    
team_names_FPAPG['G_FPAPG_PG'] = team_names_FPAPG['G_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['F_FPAPG_PG'] = team_names_FPAPG['F_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['D_FPAPG_PG'] = team_names_FPAPG['D_FPAPG']/team_names_FPAPG['gamesRemaining']
team_names_FPAPG['Gmod'] = team_names_FPAPG['G_FPAPG_PG'] - team_names_FPAPG['G_FPAPG_PG'].mean()
team_names_FPAPG['Fmod'] = team_names_FPAPG['F_FPAPG_PG'] - team_names_FPAPG['F_FPAPG_PG'].mean()
team_names_FPAPG['Dmod'] = team_names_FPAPG['D_FPAPG_PG'] - team_names_FPAPG['D_FPAPG_PG'].mean()
# team_names_FPAPG['GmodNorm'] = (team_names_FPAPG['Gmod'] - team_names_FPAPG['Gmod'].min()) / (team_names_FPAPG['Gmod'].max() - team_names_FPAPG['Gmod'].min())

team_names_FPAPG

,abbreviation,name,gamesPlayed,gamesRemaining,goalsFor,goalsAgainst,logo,conference,division,G_FPAPG,F_FPAPG,D_FPAPG,G_FPAPG_PG,F_FPAPG_PG,D_FPAPG_PG,Gmod,Fmod,Dmod
0,NYR,New York Rangers,19,63,66,45,https://assets.nhle.com/logos/nhl/svg/NYR_ligh...,Eastern,Metropolitan,122.21,1030.90,571.30,1.939841,16.363492,9.068254,0.159897,0.170936,-0.103365
1,BOS,Boston Bruins,20,62,69,50,https://assets.nhle.com/logos/nhl/svg/BOS_ligh...,Eastern,Atlantic,111.99,1011.03,566.25,1.806290,16.306935,9.133065,0.026346,0.114380,-0.038555
2,VGK,Vegas Golden Knights,21,61,69,50,https://assets.nhle.com/logos/nhl/svg/VGK_ligh...,Western,Pacific,105.81,991.69,571.84,1.734590,16.257213,9.374426,-0.045354,0.064657,0.202807
3,LAK,Los Angeles Kings,19,63,76,47,https://assets.nhle.com/logos/nhl/svg/LAK_ligh...,Western,Pacific,124.34,1032.82,591.44,1.973651,16.393968,9.387937,0.193706,0.201412,0.216317
4,VAN,Vancouver Canucks,22,60,88,56,https://assets.nhle.com/logos/nhl/svg/VAN_ligh...,Western,Pacific,107.73,960.75,549.83,1.795500,16.012500,9.163833,0.015556,-0.180056,-0.007786
5,COL,Colorado Avalanche,20,62,76,58,https://assets.nhle.com/logos/nhl/svg/COL_ligh...,Western,Central,109.42,1002.95,570.84,1.764839,16.176613,9.207097,-0.015106,-0.015943,0.035478
6,DAL,Dallas Stars,19,63,66,57,https://assets.nhle.com/logos/nhl/svg/DAL_ligh...,Western,Central,114.46,1026.86,583.91,1.816825,16.299365,9.268413,0.036881,0.106809,0.096794
7,WPG,Winnipeg Jets,20,62,71,59,https://assets.nhle.com/logos/nhl/svg/WPG_ligh...,Western,Central,118.86,1016.59,578.73,1.917097,16.396613,9.334355,0.137152,0.204057,0.162736
8,FLA,Florida Panthers,20,62,58,55,https://assets.nhle.com/logos/nhl/svg/FLA_ligh...,Eastern,Atlantic,99.37,1000.06,555.26,1.602742,16.130000,8.955806,-0.177202,-0.062556,-0.215813
9,DET,Detroit Red Wings,20,62,74,60,https://assets.nhle.com/logos/nhl/svg/DET_ligh...,Eastern,Atlantic,116.01,1007.45,562.62,1.871129,16.249194,9.074516,0.091185,0.056638,-0.097103


# Adjusted remain based on sked

In [104]:
# for index, row in player_bios.iterrows():
#     if row['position'] == 'G':
#         player_log = allG_df.loc[allG_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics_g(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPPG'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Gmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
#     elif row['position'] == 'D':
#         player_log = all_df.loc[all_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPP60'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Dmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']
#     else:
#         player_log = all_df.loc[all_df['playerId'] == row['playerId']]
#         player_log.sort_values('gameDate', inplace=True, ascending=False)
#         player_log = player_log.head(10)
#         last_10_sum = summary_statistics(player_log)
#         try:
#             last_10_FPPG = last_10_sum['FPP60'].iloc[0]
#             sos = team_names_FPAPG.loc[team_names_FPAPG['abbreviation'] == row['team']]['Fmod'].iloc[0]
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             adjRemain = (((row['FPPG'] * 2) + last_10_FPPG)/3)*row['gamesRemaining']
#             adjRemain = adjRemain + (adjRemain*sos)
#             player_bios.at[index, 'adjFantasyRemain'] = adjRemain
#         except:
#             player_bios.at[index, 'last10FPPG'] = last_10_FPPG
#             player_bios.at[index, 'adjFantasyRemain'] = row['fantasyRemain']

# player_bios.sort_values('adjFantasyRemain')

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,last10FPPG,adjFantasyRemain
817,8479406,F. Gustavsson,MIN,G,36534,11,-7.8,-0.77,-0.71,0,...,Filip Gustavsson,1998-06-07,L,74,184,64.259140,G,4272674,-2.30,-41.000222
819,8476341,A. Forsberg,OTT,G,22557,7,-7.0,-1.12,-1.00,0,...,Anton Forsberg,1992-11-27,L,75,195,1.485127,G,3036851,-1.00,-26.652907
820,8482821,A. Soderblom,CHI,G,32396,10,-9.0,-1.00,-0.90,0,...,Arvid Soderblom,1999-08-19,L,75,180,1.058999,G,4894729,-0.90,-25.325789
818,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,Jack Campbell,1992-01-09,L,75,200,2.541305,G,5473,-1.72,-21.722903
815,8470594,M. Fleury,MIN,G,32280,9,-4.6,-0.51,-0.51,0,...,Marc-Andre Fleury,1984-11-28,L,74,185,13.554605,G,2346,-0.51,-14.818995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,8475184,C. Kreider,NYR,L,21313,19,48.4,8.18,2.55,0,...,Chris Kreider,1991-04-30,L,75,233,88.900914,LW,5833,8.76,340.812687
9,8478550,A. Panarin,NYR,L,22080,19,55.2,9.00,2.91,0,...,Artemi Panarin,1991-10-30,R,72,175,98.138208,LW,3891952,8.35,348.435482
16,8476887,F. Forsberg,NSH,L,23028,20,55.4,8.66,2.77,0,...,Filip Forsberg,1994-08-13,R,73,205,89.657745,LW,2968772,11.04,368.029221
24,8478398,K. Connor,WPG,L,24800,20,53.4,7.75,2.67,0,...,Kyle Connor,1996-12-09,L,73,183,99.271610,LW,3899952,9.62,372.262325


## Add new ranking

In [38]:
for index, row in rankings.iterrows():
    try:
        rankings.at[index,'fantasyRemain'] = player_bios.loc[player_bios['espnId'] == row['espnId']]['fantasyPointsRemain'].iloc[0]
    except:
        rankings.at[index,'fantasyRemain'] = 0


rankings['rank_2023-11-27'] = rankings['fantasyRemain'].rank(ascending=False, method='min').astype(int)
rankings 

rankings = rankings[column_order]
rankings.sort_values('rank_2023-11-27')

fixes = {
    'Tim Stutzle': 'Tim Stützle',
    'Jani Hakanpaa': 'Jani Hakanpää',
    'Benoit-Olivier Groulx': 'Bo Groulx',
    'Jesse Ylonen': 'Jesse Ylönen',
    'Alexis Lafreniere': 'Alexis Lafrenière',
    'Gustav Lindstrom': 'Gustav Lindström',
    'Alexander Kerfoot': 'Alex Kerfoot',
#     'Johnny Beecher': 'John Beecher',
    'Samuel Walker': 'Sammy Walker',
    'Alex Barre-Boulet': 'Alex Barré-Boulet'
}

inverse_fixes = {value: key for key, value in fixes.items()}

rankings['FullName'].replace(fixes, inplace=True)
rankings

,espnId,FullName,fantasyRemain,rank_2023-11-27,rank_2023-11-20,rank_2023-11-13,rank_2023-11-06,rank_2023-10-30,rank_2023-10-23,rank_2023-09-01
0,4024123,Auston Matthews,199.08,5,3.0,1.0,1.0,2.0,2.0,5.0
1,3114778,David Pastrnak,200.88,4,1.0,2.0,3.0,4.0,5.0,4.0
2,3895074,Connor McDavid,164.43,25,2.0,3.0,2.0,1.0,1.0,1.0
3,4233566,Elias Pettersson,163.20,29,10.0,4.0,6.0,7.0,6.0,3.0
4,4565222,Jack Hughes,197.82,6,4.0,5.0,5.0,5.0,7.0,13.0
...,...,...,...,...,...,...,...,...,...,...
1040,5473,Jack Campbell,-28.67,1044,807.0,1041.0,763.0,369.0,260.0,369.0
1041,4565259,Pyotr Kochetkov,-8.60,1037,794.0,1042.0,762.0,213.0,260.0,213.0
1042,3151341,Kaapo Kahkonen,-11.06,1040,798.0,1043.0,761.0,1036.0,260.0,1036.0
1043,3095330,Pheonix Copley,19.76,664,808.0,1044.0,759.0,332.0,260.0,332.0


In [51]:
file_name = f"data/rankings_worksheet_{(today).strftime('%Y-%m-%d')}.xlsx"
# file_name = f"data/manual-adjustments.xlsx"
rankings.to_excel(file_name, index=False)

In [52]:
last_10['rank'] = last_10['fantasyPoints'].rank(method='first', ascending=False)
last_10['rank'] = last_10['rank'].astype(int)
last_10.head(40)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,rank
0,8475660,C. Talbot,LAK,G,25172,7,44.0,6.29,6.29,Cam Talbot,1987-07-05,L,76,200,80.380712,G,5734,1
1,8476453,N. Kucherov,TBL,R,11978,9,38.4,11.54,4.27,Nikita Kucherov,1993-06-17,L,71,182,99.876999,RW,2563060,2
2,8480069,C. Makar,COL,D,14689,10,37.1,9.09,3.71,Cale Makar,1998-10-30,R,71,187,99.907738,D,4233563,3
3,8476887,F. Forsberg,NSH,L,11439,10,35.5,11.17,3.55,Filip Forsberg,1994-08-13,R,73,205,89.145652,LW,2968772,4
4,8478499,A. Hill,VGK,G,21614,6,34.8,5.80,5.80,Adin Hill,1996-05-11,L,76,215,92.994060,G,3939714,5
5,8476468,J. Miller,VAN,C,11803,10,33.5,10.22,3.35,J.T. Miller,1993-03-14,L,73,218,99.733474,C,2590852,6
6,8477956,D. Pastrnak,BOS,R,11590,10,33.4,10.37,3.34,David Pastrnak,1996-05-25,R,72,196,99.910678,RW,3114778,7
7,8478398,K. Connor,WPG,L,11723,10,33.0,10.13,3.30,Kyle Connor,1996-12-09,L,73,183,99.270714,LW,3899952,8
8,8477501,V. Nichushkin,COL,R,13240,10,32.9,8.95,3.29,Valeri Nichushkin,1995-03-04,L,76,210,72.316434,RW,3042003,9
9,8477492,N. MacKinnon,COL,C,13517,10,32.7,8.71,3.27,Nathan MacKinnon,1995-09-01,R,72,200,99.900419,C,3041969,10


In [47]:
player_bios
remain_rank = 0
for index, row in player_bios.iterrows():
    try:
        rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-11-27'].iloc[0]
        last_rank = rankings.loc[rankings['espnId'] == row['espnId']]['rank_2023-11-20'].iloc[0]
        past_10 = last_10.loc[rankings['espnId'] == row['espnId']]['rank'].iloc[0]
    except:
        rank = 300
        last_rank = 300
        past_10 = 800
        
    remain_rank += 1
        
    player_bios.at[index, 'remain_rank'] =  remain_rank
    player_bios.at[index, 'last_rank'] =  last_rank
    player_bios.at[index, 'rank'] =  rank
    player_bios.at[index, 'past_10'] =  past_10
    
not_ranked = player_bios.loc[player_bios['rank'] == 300]
for index, row in not_ranked.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")
    if row['remain_rank'] > 300:
        break

print('-------------')        

poor_rank = player_bios.loc[(player_bios['rank'] >= 250) & (player_bios['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['remain_rank']}")


H. Thrun: 38.0
B. Biro: 58.0
J. Bernard-Docker: 113.0
C. Ingram: 130.0
N. Seeler: 188.0
N. Okhotiuk: 204.0
B. Faber: 230.0
R. Ristolainen: 234.0
K. Korczak: 251.0
D. Mermis: 260.0
P. Wotherspoon: 266.0
L. Belpedio: 277.0
A. Vlasic: 281.0
B. Malenstyn: 283.0
S. Stastney: 305.0
-------------
J. Brown: 259.0
S. Couturier: 264.0
S. Monahan: 265.0
P. Laine: 278.0
A. Henrique: 280.0
A. Lafrenière: 282.0
M. Boldy: 285.0
C. Zary: 287.0
A. Ekblad: 292.0
R. Smith: 318.0
T. Terry: 325.0
C. Hart: 326.0
N. Hischier: 330.0
A. Svechnikov: 340.0
P. Dubois: 350.0
D. Severson: 400.0
J. Binnington: 423.0
J. Gibson: 508.0
B. Montour: 575.0
J. Spurgeon: 626.0
J. Saros: 653.0
F. Gustavsson: 818.0


In [48]:
def convert_to_slug(name):
    # Convert to lowercase and replace spaces with hyphens
    slug = name.lower().replace(' ', '-')
    return slug

In [49]:
player_bios['rank_by_pos'] = player_bios.groupby('default_pos')['rank'].rank(ascending=True)
player_bios.head(50)

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,height,weight,roster_percent,default_pos,espnId,remain_rank,last_rank,rank,past_10,rank_by_pos
0,8476453,N. Kucherov,TBL,R,25780,20,73.5,10.26,3.68,0,...,71.0,182.0,99.876999,RW,2563060.0,1.0,5.0,1.0,6.0,1.0
1,8475660,C. Talbot,LAK,G,49933,14,63.2,4.56,4.51,0,...,76.0,200.0,80.380712,G,5734.0,2.0,31.0,2.0,47.0,1.0
2,8480069,C. Makar,COL,D,28971,20,65.4,8.13,3.27,0,...,71.0,187.0,99.907738,D,4233563.0,3.0,6.0,3.0,11.0,1.0
3,8477956,D. Pastrnak,BOS,R,23219,20,64.7,10.03,3.24,0,...,72.0,196.0,99.910678,RW,3114778.0,4.0,1.0,4.0,2.0,2.0
4,8479318,A. Matthews,TOR,C,24219,19,60.0,8.92,3.16,0,...,75.0,215.0,99.913606,C,4024123.0,5.0,3.0,5.0,1.0,1.0
5,8481559,J. Hughes,NJD,C,17363,14,44.0,9.12,3.14,0,...,71.0,175.0,99.692497,C,4565222.0,6.0,4.0,6.0,5.0,2.0
6,8476468,J. Miller,VAN,C,26486,22,71.9,9.77,3.27,0,...,73.0,218.0,99.733474,C,2590852.0,7.0,8.0,7.0,21.0,3.0
7,8477939,W. Nylander,TOR,R,23026,19,57.4,8.97,3.02,0,...,72.0,202.0,99.837463,RW,3114736.0,8.0,9.0,8.0,19.0,3.0
8,8477933,S. Reinhart,FLA,C,24920,20,60.0,8.67,3.00,0,...,74.0,193.0,98.635272,RW,3114722.0,9.0,28.0,9.0,36.0,4.0
9,8478550,A. Panarin,NYR,L,22080,19,55.2,9.00,2.91,0,...,72.0,175.0,98.097874,LW,3891952.0,10.0,7.0,10.0,17.0,1.0


In [8]:
# player_bios.to_excel('data/manual-adjustments.xlsx', index=False)

player_bios_adjusted = pd.read_excel('data/manual-adjustments.xlsx')

# vasil = {'playerId': 8476883, 'name': 'A. Vasilevskiy', 'team': 'TBL', 'position': 'G', 'default_pos': 'G', 'espnId': 2976847, 'remain_rank': 55, 'fullName': 'Andrei Vasilevskiy'}
# player_bios_adjusted = player_bios_adjusted.append(vasil, ignore_index=True)

for index, row in player_bios_adjusted.iterrows():
    player_bios_adjusted.at[index, 'fullName'] = player_bios.loc[player_bios['playerId'] == row['playerId'], 'fullName'].iloc[0]
    player_bios_adjusted.at[index, 'default_pos'] = player_bios.loc[player_bios['playerId'] == row['playerId'], 'default_pos'].iloc[0]
    
player_bios_adjusted['espnId'] = player_bios_adjusted['espnId'].fillna(0).astype(int)
player_bios_adjusted = player_bios_adjusted.sort_values('remain_rank', ascending=True)
player_bios_adjusted['rank'] = player_bios_adjusted['remain_rank'].rank(ascending=True, method='first')
player_bios_adjusted['rank_by_pos'] = player_bios_adjusted.groupby('default_pos')['rank'].rank(ascending=True)
player_bios_adjusted.head(60)



,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,height,weight,roster_percent,default_pos,espnId,remain_rank,last_rank,rank,past_10,rank_by_pos
3,8477956,D. Pastrnak,BOS,R,23219,20,64.7,10.03,3.24,0,...,72.0,196.0,99.910678,RW,3114778,1.0,1.0,1.0,2,1.0
4,8479318,A. Matthews,TOR,C,24219,19,60.0,8.92,3.16,0,...,75.0,215.0,99.913606,C,4024123,2.0,3.0,2.0,1,1.0
0,8476453,N. Kucherov,TBL,R,25780,20,73.5,10.26,3.68,0,...,71.0,182.0,99.876999,RW,2563060,3.0,5.0,3.0,6,2.0
24,8478402,C. McDavid,EDM,C,22230,17,44.3,7.17,2.61,0,...,73.0,194.0,99.876988,C,3895074,3.0,2.0,4.0,3,2.0
2,8480069,C. Makar,COL,D,28971,20,65.4,8.13,3.27,0,...,71.0,187.0,99.907738,D,4233563,4.0,6.0,5.0,11,1.0
5,8481559,J. Hughes,NJD,C,17363,14,44.0,9.12,3.14,0,...,71.0,175.0,99.692497,C,4565222,6.0,4.0,6.0,5,3.0
28,8480012,E. Pettersson,VAN,C,26119,22,59.9,8.26,2.72,0,...,74.0,176.0,99.853557,C,4233566,6.0,10.0,7.0,4,4.0
10,8477492,N. MacKinnon,COL,C,26312,20,58.3,7.98,2.92,0,...,72.0,200.0,99.900419,C,3041969,7.0,20.0,8.0,7,5.0
16,8478420,M. Rantanen,COL,R,26037,20,55.5,7.67,2.78,0,...,76.0,215.0,99.888703,RW,3899938,9.0,11.0,9.0,8,3.0
29,8477934,L. Draisaitl,EDM,C,24770,19,49.0,7.12,2.58,0,...,74.0,209.0,99.913602,C,3114727,9.0,12.0,10.0,9,6.0


In [78]:
espn_teams = {
    'BOS': 'Bos',
    'VGK': 'Vgk',
    'NYR': 'NYR',
    'VAN': 'Van',
    'LAK': 'LA',
    'DAL': 'Dal',
    'FLA': 'Fla',
    'COL': 'Col',
    'WPG': 'Wpg',
    'TOR': 'Tor',
    'WSH': 'Wsh',
    'CAR': 'Car',
    'TBL': 'TB',
    'PHI': 'Phi',
    'DET': 'Det',
    'ANA': 'Ana',
    'ARI': 'Ari',
    'SEA': 'Sea',
    'NJD': 'NJ',
    'STL': 'StL',
    'NYI': 'NYI',
    'OTT': 'Ott',
    'PIT': 'Pit',
    'MTL': 'Mon',
    'BUF': 'Buf',
    'CGY': 'Cgy',
    'MIN': 'Min',
    'NSH': 'Nsh',
    'CBJ': 'Cls',
    'EDM': 'Edm',
    'CHI': 'Chi',
    'SJS': 'SJ'
}

team_names_only['espn_name'] = team_names_only['abbreviation'].map(espn_teams)
team_names_only

,abbreviation,name,espn_name
0,NYR,New York Rangers,NYR
1,BOS,Boston Bruins,Bos
2,VGK,Vegas Golden Knights,Vgk
3,LAK,Los Angeles Kings,LA
4,VAN,Vancouver Canucks,Van
5,COL,Colorado Avalanche,Col
6,WPG,Winnipeg Jets,Wpg
7,DAL,Dallas Stars,Dal
8,FLA,Florida Panthers,Fla
9,TBL,Tampa Bay Lightning,TB


In [79]:
player_bios_adjusted['fullName'].replace(inverse_fixes, inplace=True)

player_bios_adjusted

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,height,weight,roster_percent,default_pos,espnId,remain_rank,last_rank,rank,past_10,rank_by_pos
3,8477956,D. Pastrnak,BOS,R,23219,20,64.7,10.03,3.24,0,...,72.0,196.0,99.910678,RW,3114778,1.0,1.0,1.0,2,1.0
4,8479318,A. Matthews,TOR,C,24219,19,60.0,8.92,3.16,0,...,75.0,215.0,99.913606,C,4024123,2.0,3.0,2.0,1,1.0
0,8476453,N. Kucherov,TBL,R,25780,20,73.5,10.26,3.68,0,...,71.0,182.0,99.876999,RW,2563060,3.0,5.0,3.0,6,2.0
24,8478402,C. McDavid,EDM,C,22230,17,44.3,7.17,2.61,0,...,73.0,194.0,99.876988,C,3895074,3.0,2.0,4.0,3,2.0
2,8480069,C. Makar,COL,D,28971,20,65.4,8.13,3.27,0,...,71.0,187.0,99.907738,D,4233563,4.0,6.0,5.0,11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,8480992,M. Chrona,SJS,G,1830,1,-5.4,-10.62,-5.40,0,...,76.0,194.0,0.016133,G,0,816.0,224.0,817.0,293,72.0
816,8470594,M. Fleury,MIN,G,32280,9,-4.6,-0.51,-0.51,0,...,74.0,185.0,13.787490,G,2346,817.0,802.0,818.0,615,73.0
818,8482821,A. Soderblom,CHI,G,28844,9,-5.8,-0.72,-0.64,0,...,75.0,180.0,0.813962,G,4894729,819.0,300.0,819.0,800,74.0
819,8475789,J. Campbell,EDM,G,16007,5,-8.6,-1.93,-1.72,0,...,75.0,200.0,2.566547,G,5473,820.0,300.0,820.0,800,75.0


In [84]:
player_bios_adjusted
poor_rank = player_bios_adjusted.loc[(player_bios_adjusted['rank'] >= 250) & (player_bios_adjusted['last_rank'] <= 250)]
for index, row in poor_rank.iterrows():
    print(f"{row['name']}: {row['last_rank']}")

F. Zetterlund: 219.0
J. Barron: 229.0
R. Colton: 233.0
T. Seguin: 246.0
N. Bjugstad: 248.0
M. Del Gaizo: 224.0
J. Brown: 244.0
M. Samuelsson: 174.0
A. Henrique: 218.0
C. Zary: 153.0
M. Pospisil: 224.0
D. Severson: 192.0
J. Gibson: 213.0
D. Hunt: 224.0
L. Karlsson: 224.0
H. Haman Aktell: 224.0
T. Luneau: 224.0
E. Andrae: 224.0
R. Jarventie: 224.0
R. Johnson: 224.0
J. Sourdif: 224.0
J. Ludvig: 224.0
H. Shepard: 224.0
M. Samoskevich: 224.0
I. Solovyov: 224.0
N. Matinpalo: 224.0
L. Rousek: 224.0
R. Lavoie: 224.0
F. Minten: 224.0
R. Winterton: 224.0
S. Malinski: 224.0
M. Lajoie: 224.0
O. Pavel: 224.0
M. Tomkins: 224.0
M. Savoie: 224.0
M. Chrona: 224.0


In [85]:
# 1. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C1) <br />

ranking_module = pd.DataFrame()

for index, row in player_bios_adjusted.iterrows():
    base_string = '<a href="http://www.espn.com/nhl/player/_/id/'
    rank = str(int(row['rank']))
    Pid = str(row['espnId']) + '/'
    name = row['fullName']
    slug = convert_to_slug(name)
    pos = row['default_pos']
    team = team_names_only.loc[team_names_only['abbreviation'] == row['team']]['espn_name'].iloc[0]
    pos_rank = pos + str(int(row['rank_by_pos']))
    
    assemble = rank + '. ' + base_string + Pid + slug + '">' + name + '</a>, ' + pos + ', ' + team + ' (' + pos_rank + ')'
    
    temp = pd.DataFrame({'column': [assemble]})
    
    ranking_module = pd.concat([ranking_module, temp])
    
    print(f"{assemble}")
    
    if int(rank) == 250:
        break


1. <a href="http://www.espn.com/nhl/player/_/id/3114778/david-pastrnak">David Pastrnak</a>, RW, Bos (RW1)
2. <a href="http://www.espn.com/nhl/player/_/id/4024123/auston-matthews">Auston Matthews</a>, C, Tor (C1)
3. <a href="http://www.espn.com/nhl/player/_/id/2563060/nikita-kucherov">Nikita Kucherov</a>, RW, TB (RW2)
4. <a href="http://www.espn.com/nhl/player/_/id/3895074/connor-mcdavid">Connor McDavid</a>, C, Edm (C2)
5. <a href="http://www.espn.com/nhl/player/_/id/4233563/cale-makar">Cale Makar</a>, D, Col (D1)
6. <a href="http://www.espn.com/nhl/player/_/id/4565222/jack-hughes">Jack Hughes</a>, C, NJ (C3)
7. <a href="http://www.espn.com/nhl/player/_/id/4233566/elias-pettersson">Elias Pettersson</a>, C, Van (C4)
8. <a href="http://www.espn.com/nhl/player/_/id/3041969/nathan-mackinnon">Nathan MacKinnon</a>, C, Col (C5)
9. <a href="http://www.espn.com/nhl/player/_/id/3899938/mikko-rantanen">Mikko Rantanen</a>, RW, Col (RW3)
10. <a href="http://www.espn.com/nhl/player/_/id/3114727/leon-

In [86]:
fileName = f"data/2024Rankings-06-{(today+timedelta(days=1)).strftime('%b%d')}.xlsx"
ranking_module.to_excel(fileName, index=False, header=False)

In [92]:
for i in range(250):
    print(f"{player_bios_adjusted.iloc[i, 1]}: {int(player_bios_adjusted.iloc[i, 21]) - int(player_bios_adjusted.iloc[i, 20])}")

D. Pastrnak: 0
A. Matthews: 1
N. Kucherov: 2
C. McDavid: -1
C. Makar: 2
J. Hughes: -2
E. Pettersson: 4
N. MacKinnon: 13
M. Rantanen: 2
L. Draisaitl: 3
A. Panarin: -3
W. Nylander: -2
Q. Hughes: 1
J. Miller: -5
N. Dobson: 0
A. Hill: 5
B. Tkachuk: 6
J. Oettinger: 11
S. Crosby: -5
C. Talbot: 10
K. Kaprizov: 4
T. Demko: -3
K. Connor: -5
B. Point: 21
J. Bratt: -6
M. Marner: 0
J. Trouba: -4
A. Fox: 15
S. Reinhart: 0
I. Shesterkin: 5
F. Forsberg: -2
E. Bouchard: 2
T. Stützle: 15
C. Kreider: 23
R. Dahlin: 9
M. Seider: 24
J. Eriksson Ek: 10
S. Stamkos: 4
C. McAvoy: -17
J. Chychrun: 0
M. Tkachuk: 16
I. Sorokin: 14
A. Pietrangelo: 5
J. Eichel: 1
A. Georgiev: 15
J. Robertson: 17
D. Hamilton: -7
J. Pavelski: 17
M. Matheson: -20
A. Kempe: -12
B. Boeser: -7
Z. Hyman: 14
M. Zibanejad: 16
A. Vasilevskiy: 251
A. DeBrincat: 20
A. Kopitar: 0
M. Rielly: -5
J. Swayman: -9
R. Hintz: 14
E. Karlsson: -4
R. O'Reilly: 0
J. Sanderson: -7
V. Hedman: 40
B. Rust: -3
D. Larkin: 32
C. Hellebuyck: 15
A. Martinez: -1
J. 